In [62]:
import os
import sys
import subprocess
import glob
import re
from datetime import datetime

# --- CONFIGURATION ---
# ▼▼▼ [필수] 분석할 로그들이 모여있는 상위 폴더 경로 ▼▼▼
LOG_BASE_PATH = "log"  # 이제 'log' 폴더를 기본 경로로 지정합니다.
# ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲

# --- (선택) 시작 및 종료 프레임 설정 ---
START_FRAME = 0 
END_FRAME = 25

# --- 스크립트 경로 (이 노트북은 src/ 상위 폴더에 있다고 가정) ---
ESTIMATOR_SCRIPT_PATH = "src/estimate_performance_our_hw.py"
ANALYZER_SCRIPT_PATH = "src/analyze_results_speed.py"
CONFIG_FILE_PATH = "src/config_merged.json"
REPORT_BASE_DIR = "report"
ANALYZER_ENG_SCRIPT_PATH = "src/analyze_results_eng.py"


# --- 1. 사전 경로 검증 (공통 파일들) ---
print("--- 🛠️ 공통 설정 검증 ---")
print(f"Python 실행 파일: {sys.executable}")
print(f"로그 상위 폴더: {os.path.abspath(LOG_BASE_PATH)}")
print(f"설정 파일: {os.path.abspath(CONFIG_FILE_PATH)}")
print(f"추정기 스크립트: {os.path.abspath(ESTIMATOR_SCRIPT_PATH)}")
print(f"분석기 스크립트: {os.path.abspath(ANALYZER_SCRIPT_PATH)}")

# 공통 파일들이 존재하는지 먼저 확인
common_paths_to_check = [
    LOG_BASE_PATH,
    CONFIG_FILE_PATH,
    ESTIMATOR_SCRIPT_PATH,
    ANALYZER_SCRIPT_PATH
]
all_paths_ok = True
for path in common_paths_to_check:
    if not os.path.exists(path):
        print(f"\n[오류!] 필수 파일/폴더를 찾을 수 없습니다: {path}")
        all_paths_ok = False

if not all_paths_ok:
    print("\n[실패] 필수 파일/폴더가 없어 스크립트를 중단합니다.")
else:
    print("\n[성공] 공통 파일 및 경로가 확인되었습니다. 개별 씬 분석을 시작합니다.")
    print("-" * 50)

    # --- 2. LOG_BASE_PATH 하위의 모든 폴더에 대해 분석 실행 ---
    # os.path.join을 사용하여 OS에 맞는 경로 구분자(/ 또는 \)를 사용합니다.
    # '*/' 패턴은 모든 하위 디렉토리를 의미합니다.
    scene_folders = glob.glob(os.path.join(LOG_BASE_PATH, "*/"))

    if not scene_folders:
        print(f"[경고] '{LOG_BASE_PATH}' 폴더 내에 분석할 하위 폴더(씬)가 없습니다.")

    for scene_path in scene_folders:
        # glob 결과에 포함될 수 있는 마지막 경로 구분자를 제거하여 깔끔하게 만듭니다.
        scene_path = scene_path.rstrip(os.sep) 
        scene_name = os.path.basename(scene_path) # 폴더 이름만 추출 (예: 'flame_steak')
        
        print(f"\n🎬 {scene_name} 씬 분석 시작...")
        
        # ** 여기가 핵심입니다 **
        # 이 부분은 원래 스크립트의 다음 셀에 있었을 '실행' 로직에 해당합니다.
        # 각 씬 폴더(scene_path)를 인자로 사용하여 추정 및 분석 스크립트를 호출합니다.
        
        # 예시: 추정기 스크립트 실행 (실제 인자 이름은 스크립트에 맞게 수정 필요)
        try:
            print(f"  [1/2] 성능 추정 스크립트 실행 중...")
            estimator_cmd = [
                sys.executable, 
                ESTIMATOR_SCRIPT_PATH,
                "--log-path", scene_path,
                "--config", CONFIG_FILE_PATH,
                # 필요 시 프레임 인자 추가
                # "--start-frame", str(START_FRAME), 
                # "--end-frame", str(END_FRAME)
            ]
            subprocess.run(estimator_cmd, check=True, capture_output=True, text=True)
            print(f"  [성공] {scene_name} 성능 추정 완료.")

            # 예시: 분석기 스크립트 실행 (실제 인자 이름은 스크립트에 맞게 수정 필요)
            print(f"  [2/2] 결과 분석 스크립트 실행 중...")
            analyzer_cmd = [
                sys.executable,
                ANALYZER_SCRIPT_PATH,
                "--result-path", scene_path, # 추정 결과가 같은 폴더에 저장된다고 가정
                "--output-dir", os.path.join(REPORT_BASE_DIR, scene_name)
            ]
            subprocess.run(analyzer_cmd, check=True, capture_output=True, text=True)
            print(f"  [성공] {scene_name} 결과 분석 및 리포트 생성 완료.")

        except subprocess.CalledProcessError as e:
            print(f"  [오류!] {scene_name} 씬 처리 중 오류 발생!")
            print(f"  명령어: {' '.join(e.cmd)}")
            print(f"  에러 메시지:\n{e.stderr}")
            # 하나의 씬에서 오류가 나도 다음 씬으로 넘어가도록 continue 사용
            continue 
            
    print("\n✨ 모든 씬에 대한 분석이 완료되었습니다.")

--- 🛠️ 설정 검증 ---
Python 실행 파일: /data/yeonggeonkim/.conda/envs/4d/bin/python
로그 폴더: /data/yeonggeonkim/anaconda/cf/4dgs/4d-gaussian-splatting/sim/log/flame_steak
시작 프레임: 0
종료 프레임: 3
설정 파일: /data/yeonggeonkim/anaconda/cf/4dgs/4d-gaussian-splatting/sim/src/config_merged.json
추정기 스크립트: /data/yeonggeonkim/anaconda/cf/4dgs/4d-gaussian-splatting/sim/src/estimate_performance_our_hw.py
분석기 스크립트: /data/yeonggeonkim/anaconda/cf/4dgs/4d-gaussian-splatting/sim/src/analyze_results_speed.py

[성공] 모든 파일 및 경로가 확인되었습니다. 다음 셀을 실행하세요.


In [63]:
import re # 정규식 모듈 임포트
import os # 파일 경로 처리를 위해 임포트

print(f"--- 📊 1단계: 성능 추정기 실행 ---")
print(f"대상 폴더: {LOG_SCENE_PATH}\n")

# 1. 실행할 명령어 구성
command = [
    sys.executable,  # 현재 주피터 커널의 파이썬 사용
    ESTIMATOR_SCRIPT_PATH,
    "--path", LOG_SCENE_PATH
]

# --- 시작 및 종료 프레임 인자 추가 ---
if START_FRAME is not None:
    command.extend(["--start", str(START_FRAME)])
if END_FRAME is not None:
    command.extend(["--end", str(END_FRAME)])
# --- ---

print(f"실행 명령어: {' '.join(command)}\n")

# 2. 성능 추정 스크립트 실행
result = subprocess.run(command, capture_output=True, text=True, encoding='utf-8')

# 3. 1단계(추정기)의 전체 출력 표시
print("--- [1. Estimator Output START] ---")
print(result.stdout)
print("--- [1. Estimator Output END] ---")

# 4. 1단계 실행 결과 확인
if result.returncode != 0:
    print(f"[오류!] 1단계 성능 추정기 실행에 실패했습니다.")
    print(result.stderr)
else:
    print(f"\n[성공] 1단계 성능 추정이 완료되었습니다.")
    
    # --- 🔍 2단계: 1단계 출력에서 CSV 경로 파싱 ---
    print("\n--- 🔍 2단계: 생성된 CSV 경로 파싱 중 ---")
    csv_file_path = None
    
    # 1단계 출력(result.stdout)에서 CSV 경로를 찾습니다.
    match = re.search(r"\[\+\] Detailed data for analysis saved to: (.*)", result.stdout)
    
    if match and match.group(1):
        csv_file_path = match.group(1).strip()
        print(f"[성공] 분석 대상 CSV 찾음: {csv_file_path}")
        
        if not os.path.exists(csv_file_path):
             print(f"[경고!] 경로를 찾았으나 파일이 존재하지 않습니다: {csv_file_path}")
             csv_file_path = None # 경로를 무효화
    
    if not csv_file_path:
        print(f"[오류!] 1단계 출력에서 CSV 경로를 찾지 못했습니다. 분석을 건너뜁니다.")
        
    # --- 🚀 3단계: 스피드업 분석기 실행 ---
if csv_file_path:
    print(f"\n--- 🚀 3단계: 스피드업 분석기 즉시 실행 ---")
    
    command_analyzer = [
        sys.executable,
        ANALYZER_SCRIPT_PATH,
        csv_file_path,
        "--config", CONFIG_FILE_PATH
    ]
    
    result_analyzer = subprocess.run(command_analyzer, capture_output=True, text=True, encoding='utf-8')

    # 3단계(분석기)의 출력 표시
    print("\n--- [3. Analyzer Output START] ---")
    print(result_analyzer.stdout)
    print("--- [3. Analyzer Output END] ---")

    if result_analyzer.returncode != 0:
        print(f"[오류!] 3단계 분석 스크립트 실행에 실패했습니다.")
        print(result_analyzer.stderr)
    else:
        # 3단계(분석기)의 stdout을 텍스트 파일로 저장
        try:
            report_dir = os.path.dirname(csv_file_path)
            analysis_log_path = os.path.join(report_dir, "analysis_speedup_report.txt")
            
            with open(analysis_log_path, 'w', encoding='utf-8') as f:
                f.write(result_analyzer.stdout)
            
            print(f"\n[성공] 3단계 분석 결과를 다음 파일에 저장했습니다:\n{analysis_log_path}")
        except Exception as e:
            print(f"\n[오류!] 3단계 분석 결과 로그 파일 저장 중 오류 발생: {e}")

        # ▼▼▼▼▼ [추가된 부분] ▼▼▼▼▼
        # --- 🚀 4단계: 영문 스피드업 분석기 실행 ---
        print(f"\n--- 🚀 4단계: 영문 스피드업 분석기 실행 (analyze_results_eng.py) ---")

        command_analyzer_eng = [
            sys.executable,
            ANALYZER_ENG_SCRIPT_PATH,  # 영문 분석기 스크립트 경로 사용
            csv_file_path
        ]
        
        result_analyzer_eng = subprocess.run(command_analyzer_eng, capture_output=True, text=True, encoding='utf-8')

        # 4단계(영문 분석기)의 출력 표시
        print("\n--- [4. English Analyzer Output START] ---")
        print(result_analyzer_eng.stdout)
        print("--- [4. English Analyzer Output END] ---")

        if result_analyzer_eng.returncode != 0:
            print(f"[오류!] 4단계 영문 분석 스크립트 실행에 실패했습니다.")
            print(result_analyzer_eng.stderr)
        else:
            # 4단계(영문 분석기)의 stdout을 텍스트 파일로 저장
            try:
                # CSV 파일이 저장된 동일한 디렉토리를 사용 (report_dir 변수는 이미 존재)
                analysis_log_eng_path = os.path.join(report_dir, "analysis_speedup_report_eng.txt")
                
                with open(analysis_log_eng_path, 'w', encoding='utf-8') as f:
                    f.write(result_analyzer_eng.stdout)
                
                print(f"\n[성공] 4단계 영문 분석 결과를 다음 파일에 저장했습니다:\n{analysis_log_eng_path}")
            except Exception as e:
                print(f"\n[오류!] 4단계 영문 분석 결과 로그 파일 저장 중 오류 발생: {e}")
        # ▲▲▲▲▲ [추가된 부분] ▲▲▲▲▲

--- 📊 1단계: 성능 추정기 실행 ---
대상 폴더: log/flame_steak

실행 명령어: /data/yeonggeonkim/.conda/envs/4d/bin/python src/estimate_performance_our_hw.py --path log/flame_steak --start 0 --end 3

--- [1. Estimator Output START] ---
[*] Analyzing logs from: /data/yeonggeonkim/anaconda/cf/4dgs/4d-gaussian-splatting/sim/log/flame_steak

 GSCore Performance Estimator (Analyzing Frames 0 to 3)
1
2
0
0
3
3
2
2
1
3
0
2
1
0
0
0
1
2
1
3
4
1
3
2
4
3
4
4
5
2
2
4
7
5
3
2
4
1
5
4
3
3
1
6
1
4
3
2
6
3
7
5
6
3
5
7
2
5
3
7
1
3
5
3
8
3
6
3
6
4
6
2
7
4
3
9
5
8
4
8
4
7
6
8
5
7
4
7
4
6
5
6
6
9
4
7
4
1
10
6
1
4
4
8
3
8
3
1
3
2
9
2
4
2
9
0
5
7
2
0
3
7
2
4
9
1
1
4
8
1
1
5
8
1
7
6
4
1
9
5
3
8
9
1
8
7
1
2
8
5
1
8
5
3
10
2
8
3
7
7
3
6
7
2
8
7
2
6
3
9
6
4
8
5
0
6
6
1
6
5
2
3
8
4
3
1
6
1
4
6
3
3
6
2
4
5
2
1
5
4
1
2
2
8
2
6
1
1
7
0
2
1
6
1
1
5
1
2
5
3
1
5
2
4
1
6
2
3
5
3
25

3
7
2
3
5
3
4
5
2
2
6
4
1
4
5
4
6
3
4
3
6
4
2
6
4
2
6
4
6
4
3
5
3
5
9
4
3
6
2
3
3
9
4
6
8
3
3
4
4
2
8
3
2
5
2
4
3
4
5
4
4
4
3
4
3
4
6
3
6
2
5
5
6
3
3
6
4
3
8
5